In [1]:
url = 'https://m.joongna.com/search-list/product?searchword=맥북%20프로'

#### 01. BeautifulSoup (can't load html code)

In [4]:
import requests
from bs4 import BeautifulSoup

response = requests.get(url)

#### 02. scrapy (can't load html code)

In [5]:
import scrapy
from scrapy.http import TextResponse

req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding='utf-8')

#### 03. selenium

In [6]:
def joongonara():
    from selenium import webdriver
    from fake_useragent import UserAgent

    url = 'https://m.joongna.com/search-list/product?searchword={}%20{}'.format("맥북", "프로")
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent={}".format(UserAgent().chrome))
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    links = driver.find_elements_by_css_selector('#root > div.page.pd_t110 > div.pd_b20 > div.SearchList_listWrap__14Cu9.pd_t43 > div > div > a')
    df = []

    for link in links:
        link = link.get_attribute('href')
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("user-agent={}".format(UserAgent().chrome))
        driver = webdriver.Chrome(options=options)
        driver.get(link)
        try:
            title = driver.find_elements_by_css_selector('.pd20 > p')[0].text
        except:
            title = None
        try:
            price = driver.find_elements_by_css_selector('.pd20 strong')[0].text
        except:
            price = 0
        try:
            region = driver.find_elements_by_css_selector('button .f15')[0].text
        except:
            region = None
        try:    
            view_count = driver.find_elements_by_css_selector('p .c_orange')[0].text
        except:
            view_count = 0
        try:
            desc = driver.find_elements_by_css_selector('.ProductDetailComponent_tag__1sc7f.mb8')[0].text
        except:
            desc = None
        df.append({'title' : title, 'price' : price, 'region' : region, 'view_count' : view_count, 'desc' : desc})
        driver.quit()
    return pd.DataFrame(df)


In [7]:
joogonara_df = joongonara()

In [8]:
joogonara_df.tail(2)

,title,price,region,view_count,desc
38,2019 맥북 프로 13인치 스페이스그레이 256,"1,100,000원",None,27,거의새것
39,None,0,None,0,None
